In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
#from yelp.client import Client
import json
from ratelimit import limits
import requests
import csv
import random
from lock import yelp_key
my_api_key = yelp_key


df = pd.read_csv("../sub-est2019_all.csv", engine='python')
rural_city = df[(df["POPESTIMATE2019"]< 50000) & (df["POPESTIMATE2019"] > 1000) & (df["PRIMGEO_FLAG"]==0)]
rural_df = rural_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_city = df[df["POPESTIMATE2019"]>= 50000]
urban_df = urban_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_index = urban_df.loc[urban_df["NAME"] == urban_df["STNAME"]].index
urban_clean = urban_df.drop(urban_index)
random_urban_city = urban_clean[["NAME", "STNAME","POPESTIMATE2019"]].sample(n=100, random_state=1)
random_rural_city = rural_df[["NAME", "STNAME", "POPESTIMATE2019"]].sample(n=100, random_state=1)


In [ ]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

rural_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
#     "Category 2": [],
#     "Category 3": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
offset = np.arange(0,950,50).tolist() 

for city, state in zip(random_rural_city["NAME"], random_rural_city["STNAME"]):
    for i in range(len(offset)):
        params = {
            'location': f"{city}, {state}",
            'radius': 8047,
            'limit': 50,
            'offset': offset[i]
                 }
        response = requests.get(url, params = params, headers = headers).json()
        try:
            for business in range(len(response["businesses"])):
                try:
                    rural_dict["Searched"].append(f"{city}, {state}")
                    rural_dict["City"].append(response["businesses"][business]["location"]["city"])
                    rural_dict["State"].append(response["businesses"][business]["location"]["state"])
                    rural_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
                    rural_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
                    rural_dict["Restaurant"].append(response["businesses"][business]["name"])
                    rural_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
#                     rural_dict["Category"].append(response["businesses"][business]["categories"][0]["title"])
#                     rural_dict["Category 2"].append(response["businesses"][business]["categories"][1]["title"])
#                     rural_dict["Category 3"].append(response["businesses"][business]["categories"][2]["title"])
                    rural_dict["Transactions"].append(response["businesses"][business]["transactions"])
                except KeyError:
                    rural_dict["Searched"].append("No City Found")
                    rural_dict["City"].append("No City Found")
                    rural_dict["State"].append("No State Found")
                    rural_dict["Lat"].append("No Lat Found")
                    rural_dict["Lng"].append("No Lng Found")
                    rural_dict["Restaurant"].append("No Restaurant Found")
                    rural_dict["Category"].append("No Categories")
                    rural_dict["Transactions"].append("No Transactions Found")
#                 except IndexError:
#                     rural_dict["Category 2"].append("")
#                     rural_dict["Category 3"].append("")
        except KeyError:
                pass

In [ ]:
rural = pd.DataFrame.from_dict(rural_dict, orient='index')
rural = rural.transpose()
rural.head(70)

In [2]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

urban_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
#     "Category 2": [],
#     "Category 3": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
offset = np.arange(0,950,50).tolist() 

for city, state in zip(random_urban_city["NAME"], random_urban_city["STNAME"]):
    for i in range(len(offset)):
        params = {
            'location': f"{city}, {state}",
            'radius': 8047,
            'limit': 50,
            'offset': offset[i]
                 }
        response = requests.get(url, params = params, headers = headers).json()
        try:
            for business in range(len(response["businesses"])):
                try:
                    urban_dict["Searched"].append(f"{city}, {state}")
                    urban_dict["City"].append(response["businesses"][business]["location"]["city"])
                    urban_dict["State"].append(response["businesses"][business]["location"]["state"])
                    urban_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
                    urban_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
                    urban_dict["Restaurant"].append(response["businesses"][business]["name"])
                    urban_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
#                     rural_dict["Category"].append(response["businesses"][business]["categories"][0]["title"])
#                     rural_dict["Category 2"].append(response["businesses"][business]["categories"][1]["title"])
#                     rural_dict["Category 3"].append(response["businesses"][business]["categories"][2]["title"])
                    urban_dict["Transactions"].append(response["businesses"][business]["transactions"])
                except KeyError:
                    urban_dict["Searched"].append("No City Found")
                    urban_dict["City"].append("No City Found")
                    urban_dict["State"].append("No State Found")
                    urban_dict["Lat"].append("No Lat Found")
                    urban_dict["Lng"].append("No Lng Found")
                    urban_dict["Restaurant"].append("No Restaurant Found")
                    urban_dict["Category"].append("No Categories")
                    urban_dict["Transactions"].append("No Transactions Found")
#                 except IndexError:
#                     rural_dict["Category 2"].append("")
#                     rural_dict["Category 3"].append("")
        except KeyError:
                pass

In [3]:
urban = pd.DataFrame.from_dict(urban_dict, orient='index')
urban = urban.transpose()
urban.head()

,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions


In [4]:
urban.to_csv("../urbantest.csv")
#rural.to_csv("../ruraltest.csv")

In [5]:
rural_cat= pd.DataFrame(rural["Category"].to_list(), columns = ["Category 1", "Category 2", "Category 3", "Category 4", "Category 5", "Category 6"])
rural_clean = rural_cat[["Category 1", "Category 2", "Category 3"]]
rural_clean = rural.merge(rural_cat, left_index = True,right_index = True)
rural_clean.head()

NameError: name 'rural' is not defined

In [ ]:
urban_cat= pd.DataFrame(urban["Category"].to_list(), columns = ["Category 1", "Category 2", "Category 3", "Category 4"], index = urban.index)
urban_cats = urban_cat[["Category 1", "Category 2", "Category 3"]]
urban_clean = urban.merge(urban_cats, left_index = True,right_index = True)
urban_clean.head(30)

In [37]:
path = (r"C:\Users\yenia\Desktop\JHU_Bootcamp\Project_1\Project_one\Data Pull\urban_clean.csv")
urban_clean = pd.read_csv(path)
urban_clean

,Unnamed: 0,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions,Category 1,Category 2,Category 3
0,0,"Spokane Valley city, Washington",Spokane Valley,WA,47.656965,-117.278017,La Plaza De Mexico,['Mexican'],['delivery'],Mexican,NaN,NaN
1,1,"Spokane Valley city, Washington",Spokane Valley,WA,47.657224,-117.261841,Choo Choo Tortas,['Mexican'],['delivery'],Mexican,NaN,NaN
2,2,"Spokane Valley city, Washington",Spokane Valley,WA,47.613473,-117.243869,Craft & Gather,"['American (Traditional)', 'Bars']",[],American (Traditional),Bars,NaN
3,3,"Spokane Valley city, Washington",Spokane Valley,WA,47.666632,-117.239067,Shamus's Sandwich Shop,['Sandwiches'],['delivery'],Sandwiches,NaN,NaN
4,4,"Spokane Valley city, Washington",Spokane Valley,WA,47.662079,-117.240514,Little Euro,"['Breakfast & Brunch', 'Coffee & Tea']",['delivery'],Breakfast & Brunch,Coffee & Tea,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
47151,47151,"Peoria city, Arizona",Maricopa,AZ,33.068770,-112.002080,Maricopa Outdoor Adventures,"['Hiking', 'Tours', 'Rock Climbing']",[],Hiking,Tours,Rock Climbing
47152,47152,"Peoria city, Arizona",Phoenix,AZ,33.453766,-112.073967,Tropic Like Its Hot,"['Food Trucks', 'Desserts', 'Burgers']",[],Food Trucks,Desserts,Burgers
47153,47153,"Peoria city, Arizona",Scottsdale,AZ,33.510676,-111.895378,Mobile Marinara,"['Chicken Wings', 'Italian', 'Food Trucks']",[],Chicken Wings,Italian,Food Trucks
47154,47154,"Peoria city, Arizona",San Tan Valley,AZ,33.151720,-111.563040,Nonla Viet,"['Vietnamese', 'Food Trucks']",[],Vietnamese,Food Trucks,NaN


In [42]:
urban_clean_opt = urban_clean
urban_clean_opt["Vegan Options"] = ""
urban_clean_opt.head()
urban_clean_opt["Vegan Options"] = np.where(urban_clean["Category 1"].isin(keywords)|urban_clean["Category 2"].isin(keywords)|urban_clean["Category 3"].isin(keywords), 1, 0)
urban_clean_opt

,Unnamed: 0,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions,Category 1,Category 2,Category 3,Vegan Options
0,0,"Spokane Valley city, Washington",Spokane Valley,WA,47.656965,-117.278017,La Plaza De Mexico,['Mexican'],['delivery'],Mexican,NaN,NaN,0
1,1,"Spokane Valley city, Washington",Spokane Valley,WA,47.657224,-117.261841,Choo Choo Tortas,['Mexican'],['delivery'],Mexican,NaN,NaN,0
2,2,"Spokane Valley city, Washington",Spokane Valley,WA,47.613473,-117.243869,Craft & Gather,"['American (Traditional)', 'Bars']",[],American (Traditional),Bars,NaN,0
3,3,"Spokane Valley city, Washington",Spokane Valley,WA,47.666632,-117.239067,Shamus's Sandwich Shop,['Sandwiches'],['delivery'],Sandwiches,NaN,NaN,0
4,4,"Spokane Valley city, Washington",Spokane Valley,WA,47.662079,-117.240514,Little Euro,"['Breakfast & Brunch', 'Coffee & Tea']",['delivery'],Breakfast & Brunch,Coffee & Tea,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47151,47151,"Peoria city, Arizona",Maricopa,AZ,33.068770,-112.002080,Maricopa Outdoor Adventures,"['Hiking', 'Tours', 'Rock Climbing']",[],Hiking,Tours,Rock Climbing,0
47152,47152,"Peoria city, Arizona",Phoenix,AZ,33.453766,-112.073967,Tropic Like Its Hot,"['Food Trucks', 'Desserts', 'Burgers']",[],Food Trucks,Desserts,Burgers,0
47153,47153,"Peoria city, Arizona",Scottsdale,AZ,33.510676,-111.895378,Mobile Marinara,"['Chicken Wings', 'Italian', 'Food Trucks']",[],Chicken Wings,Italian,Food Trucks,0
47154,47154,"Peoria city, Arizona",San Tan Valley,AZ,33.151720,-111.563040,Nonla Viet,"['Vietnamese', 'Food Trucks']",[],Vietnamese,Food Trucks,NaN,0


In [46]:
urban_vegan_opt = urban_clean.loc[urban_clean["Vegan Options"] == 1]
urban_vegan_opt

,Unnamed: 0,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions,Category 1,Category 2,Category 3,Vegan Options
321,321,"Oceanside city, California",Oceanside,CA,33.175745,-117.363002,The Plot,"['Vegan', 'American (New)']",['delivery'],Vegan,American (New),NaN,1
340,340,"Oceanside city, California",Oceanside,CA,33.196250,-117.379320,Rim Talay Thai Cuisine,"['Thai', 'Vegan', 'Gluten-Free']",['delivery'],Thai,Vegan,Gluten-Free,1
408,408,"Oceanside city, California",Oceanside,CA,33.183085,-117.331893,Everbowl,"['Acai Bowls', 'Juice Bars & Smoothies', 'Vegan']","['pickup', 'delivery']",Acai Bowls,Juice Bars & Smoothies,Vegan,1
923,923,"Oceanside city, California",Carlsbad,CA,33.138800,-117.272360,Plantiful Kitchen,"['Bakeries', 'Vegan', 'Desserts']",[],Bakeries,Vegan,Desserts,1
985,985,"Oceanside city, California",Carlsbad,CA,33.139909,-117.272801,Choice Juicery,"['Vegan', 'Juice Bars & Smoothies']","['delivery', 'pickup']",Vegan,Juice Bars & Smoothies,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45722,45722,"Perth Amboy city, New Jersey",Edison,NJ,40.547891,-74.334781,Potatopia,"['Fast Food', 'American (Traditional)', 'Vegan']","['pickup', 'delivery']",Fast Food,American (Traditional),Vegan,1
45773,45773,"Perth Amboy city, New Jersey",Staten Island,NY,40.522221,-74.239082,Love Earth,"['Bakeries', 'Vegan']","['delivery', 'pickup']",Bakeries,Vegan,NaN,1
46397,46397,"Perth Amboy city, New Jersey",Millstone,NJ,40.205016,-74.448224,Try Vegan,"['Vegan', 'Juice Bars & Smoothies', 'Food Truc...",[],Vegan,Juice Bars & Smoothies,Food Trucks,1
46479,46479,"Peoria city, Arizona",Peoria,AZ,33.596871,-112.203181,Casa De Falafel,"['Mediterranean', 'Vegan', 'Middle Eastern']",['delivery'],Mediterranean,Vegan,Middle Eastern,1


In [47]:
urban_vegan_opt.count()

Unnamed: 0       331
Searched         331
City             331
State            331
Lat              331
Lng              331
Restaurant       331
Category         331
Transactions     331
Category 1       331
Category 2       317
Category 3       270
Vegan Options    331
dtype: int64

In [9]:
total_all = len(urban_clean["Searched"])
print(total_all)

47156


In [10]:
#vegan = urban_clean[list(urban_clean.loc[:,"Category 1": "Category 3"] == "Vegan")]
#vegan_1 = urban_clean[urban_clean['Category 1'] == 'Vegan']
#df[ list(df.loc[:,'A':'C']) + ['E'] + list(df.loc[:,'G':'I']) ]
keywords = ["Vegan", "vegan"]
urban_vegan = urban_clean[(urban_clean["Category 1"].isin(keywords))|(urban_clean["Category 2"].isin(keywords))|(urban_clean["Category 3"].isin(keywords))]
urban_vegan_df = pd.DataFrame(urban_vegan)
urban_vegan_df.head()

,Unnamed: 0,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions,Category 1,Category 2,Category 3
321,321,"Oceanside city, California",Oceanside,CA,33.175745,-117.363002,The Plot,"['Vegan', 'American (New)']",['delivery'],Vegan,American (New),NaN
340,340,"Oceanside city, California",Oceanside,CA,33.196250,-117.379320,Rim Talay Thai Cuisine,"['Thai', 'Vegan', 'Gluten-Free']",['delivery'],Thai,Vegan,Gluten-Free
408,408,"Oceanside city, California",Oceanside,CA,33.183085,-117.331893,Everbowl,"['Acai Bowls', 'Juice Bars & Smoothies', 'Vegan']","['pickup', 'delivery']",Acai Bowls,Juice Bars & Smoothies,Vegan
923,923,"Oceanside city, California",Carlsbad,CA,33.138800,-117.272360,Plantiful Kitchen,"['Bakeries', 'Vegan', 'Desserts']",[],Bakeries,Vegan,Desserts
985,985,"Oceanside city, California",Carlsbad,CA,33.139909,-117.272801,Choice Juicery,"['Vegan', 'Juice Bars & Smoothies']","['delivery', 'pickup']",Vegan,Juice Bars & Smoothies,NaN


In [11]:
urban_vegan_df.count()

Unnamed: 0      331
Searched        331
City            331
State           331
Lat             331
Lng             331
Restaurant      331
Category        331
Transactions    331
Category 1      331
Category 2      317
Category 3      270
dtype: int64

In [12]:
urban_vegan_df.describe()

,Unnamed: 0,Lat,Lng
count,331.000000,331.000000,331.000000
mean,26477.069486,35.449038,-96.655201
std,13271.358055,5.256771,20.021570
min,321.000000,25.736019,-122.492114
25%,14598.500000,33.201742,-118.144691
50%,28508.000000,36.131644,-88.208500
75%,38279.000000,39.726137,-80.136308
max,47135.000000,43.095812,-70.879084


In [38]:
urban_vegan_opt = urban_vegan_df
urban_vegan_opt["Vegan Options"] = ""
urban_vegan_opt.head()
urban_vegan_opt["Vegan Options"] = np.where(urban_vegan["Category 1"].isin(keywords)|urban_vegan["Category 2"].isin(keywords)|urban_vegan["Category 3"].isin(keywords), True, False)
urban_vegan_opt

,Unnamed: 0,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions,Category 1,Category 2,Category 3,Vegan Options
321,321,"Oceanside city, California",Oceanside,CA,33.175745,-117.363002,The Plot,"['Vegan', 'American (New)']",['delivery'],Vegan,American (New),NaN,True
340,340,"Oceanside city, California",Oceanside,CA,33.196250,-117.379320,Rim Talay Thai Cuisine,"['Thai', 'Vegan', 'Gluten-Free']",['delivery'],Thai,Vegan,Gluten-Free,True
408,408,"Oceanside city, California",Oceanside,CA,33.183085,-117.331893,Everbowl,"['Acai Bowls', 'Juice Bars & Smoothies', 'Vegan']","['pickup', 'delivery']",Acai Bowls,Juice Bars & Smoothies,Vegan,True
923,923,"Oceanside city, California",Carlsbad,CA,33.138800,-117.272360,Plantiful Kitchen,"['Bakeries', 'Vegan', 'Desserts']",[],Bakeries,Vegan,Desserts,True
985,985,"Oceanside city, California",Carlsbad,CA,33.139909,-117.272801,Choice Juicery,"['Vegan', 'Juice Bars & Smoothies']","['delivery', 'pickup']",Vegan,Juice Bars & Smoothies,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45722,45722,"Perth Amboy city, New Jersey",Edison,NJ,40.547891,-74.334781,Potatopia,"['Fast Food', 'American (Traditional)', 'Vegan']","['pickup', 'delivery']",Fast Food,American (Traditional),Vegan,True
45773,45773,"Perth Amboy city, New Jersey",Staten Island,NY,40.522221,-74.239082,Love Earth,"['Bakeries', 'Vegan']","['delivery', 'pickup']",Bakeries,Vegan,NaN,True
46397,46397,"Perth Amboy city, New Jersey",Millstone,NJ,40.205016,-74.448224,Try Vegan,"['Vegan', 'Juice Bars & Smoothies', 'Food Truc...",[],Vegan,Juice Bars & Smoothies,Food Trucks,True
46479,46479,"Peoria city, Arizona",Peoria,AZ,33.596871,-112.203181,Casa De Falafel,"['Mediterranean', 'Vegan', 'Middle Eastern']",['delivery'],Mediterranean,Vegan,Middle Eastern,True


In [13]:
total_urban_vegan = len(urban_vegan["Searched"])
total_urban_vegan

331

In [35]:
urban_clean_opt.count()

Unnamed: 0       331
Searched         331
City             331
State            331
Lat              331
Lng              331
Restaurant       331
Category         331
Transactions     331
Category 1       331
Category 2       317
Category 3       270
Vegan Options    331
dtype: int64

In [36]:
urban_grouped= urban_clean_opt.groupby("Searched").count()
urban_grouped

,Unnamed: 0,City,State,Lat,Lng,Restaurant,Category,Transactions,Category 1,Category 2,Category 3,Vegan Options
Searched,,,,,,,,,,,,
"Alameda city, California",17,17,17,17,17,17,17,17,17,16,14,17
"Arcadia city, California",6,6,6,6,6,6,6,6,6,5,5,6
"Bellflower city, California",7,7,7,7,7,7,7,7,7,7,7,7
"Berkeley city, California",27,27,27,27,27,27,27,27,27,27,22,27
"Bradenton city, Florida",1,1,1,1,1,1,1,1,1,1,1,1
"Brookhaven city, Georgia",9,9,9,9,9,9,9,9,9,7,7,9
"Cambridge city, Massachusetts",25,25,25,25,25,25,25,25,25,23,18,25
"Carmel city, Indiana",2,2,2,2,2,2,2,2,2,2,2,2
"Champaign city, Illinois",2,2,2,2,2,2,2,2,2,2,2,2


In [29]:
urban_count_df = pd.DataFrame(urban_grouped_count)
del urban_count_df["Unnamed: 0"]
urban_count_df

,City,State,Lat,Lng,Restaurant,Category,Transactions,Category 1,Category 2,Category 3,Vegan Options
Searched,,,,,,,,,,,
"Alameda city, California",17,17,17,17,17,17,17,17,16,14,17
"Arcadia city, California",6,6,6,6,6,6,6,6,5,5,6
"Bellflower city, California",7,7,7,7,7,7,7,7,7,7,7
"Berkeley city, California",27,27,27,27,27,27,27,27,27,22,27
"Bradenton city, Florida",1,1,1,1,1,1,1,1,1,1,1
"Brookhaven city, Georgia",9,9,9,9,9,9,9,9,7,7,9
"Cambridge city, Massachusetts",25,25,25,25,25,25,25,25,23,18,25
"Carmel city, Indiana",2,2,2,2,2,2,2,2,2,2,2
"Champaign city, Illinois",2,2,2,2,2,2,2,2,2,2,2


In [32]:
urban_count_df.to_csv("../urbangroupedcount.csv")

In [19]:
urban_count_df.describe()

,Unnamed: 0,City,State,Lat,Lng,Restaurant,Category,Transactions,Category 1,Category 2,Category 3,Vegan Options
count,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,6.129630,6.129630,6.129630,6.129630,6.129630,6.129630,6.129630,6.129630,6.129630,5.870370,5.000000,6.129630
std,6.381123,6.381123,6.381123,6.381123,6.381123,6.381123,6.381123,6.381123,6.381123,6.136947,5.154225,6.381123
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
25%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.250000,2.000000
50%,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000
75%,7.750000,7.750000,7.750000,7.750000,7.750000,7.750000,7.750000,7.750000,7.750000,7.000000,7.000000,7.750000
max,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,22.000000,27.000000


In [20]:
total_vegan = sum(urban_count_df["Vegan Options"])
total_vegan


331

In [25]:
df["Searched"] = df["NAME"] + ", " + df["STNAME"]

#rural_clean_pop = pd.merge(rural_clean, df[["POPESTIMATE2019", "Searched"]],  how='left', on= "Searched")
urban_clean_pop = pd.merge(urban_clean, df[["POPESTIMATE2019", "Searched"]],  how='left', on= "Searched")
urban_clean_pop

,Unnamed: 0,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions,Category 1,Category 2,Category 3,Vegan Options,POPESTIMATE2019
0,0,"Spokane Valley city, Washington",Spokane Valley,WA,47.656965,-117.278017,La Plaza De Mexico,['Mexican'],['delivery'],Mexican,NaN,NaN,,101060
1,0,"Spokane Valley city, Washington",Spokane Valley,WA,47.656965,-117.278017,La Plaza De Mexico,['Mexican'],['delivery'],Mexican,NaN,NaN,,101060
2,1,"Spokane Valley city, Washington",Spokane Valley,WA,47.657224,-117.261841,Choo Choo Tortas,['Mexican'],['delivery'],Mexican,NaN,NaN,,101060
3,1,"Spokane Valley city, Washington",Spokane Valley,WA,47.657224,-117.261841,Choo Choo Tortas,['Mexican'],['delivery'],Mexican,NaN,NaN,,101060
4,2,"Spokane Valley city, Washington",Spokane Valley,WA,47.613473,-117.243869,Craft & Gather,"['American (Traditional)', 'Bars']",[],American (Traditional),Bars,NaN,,101060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112628,47151,"Peoria city, Arizona",Maricopa,AZ,33.068770,-112.002080,Maricopa Outdoor Adventures,"['Hiking', 'Tours', 'Rock Climbing']",[],Hiking,Tours,Rock Climbing,,175961
112629,47152,"Peoria city, Arizona",Phoenix,AZ,33.453766,-112.073967,Tropic Like Its Hot,"['Food Trucks', 'Desserts', 'Burgers']",[],Food Trucks,Desserts,Burgers,,175961
112630,47153,"Peoria city, Arizona",Scottsdale,AZ,33.510676,-111.895378,Mobile Marinara,"['Chicken Wings', 'Italian', 'Food Trucks']",[],Chicken Wings,Italian,Food Trucks,,175961
112631,47154,"Peoria city, Arizona",San Tan Valley,AZ,33.151720,-111.563040,Nonla Viet,"['Vietnamese', 'Food Trucks']",[],Vietnamese,Food Trucks,NaN,,175961
